In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "VietAI/envit5-translation"  # or one of the other names above

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

text = "How can I help you today, my friend?"
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
outputs = model.generate(**inputs)

translated = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(translated)  # Should print Vietnamese translation


vi: Hôm nay tôi giúp gì được cho anh đây, anh bạn?


                                              0.0/84.1 kB ? eta -:--:--
     ---------------------------------------- 84.1/84.1 kB ? eta 0:00:00
                                              0.0/104.1 kB ? eta -:--:--
     -------------------------------------- 104.1/104.1 kB 6.3 MB/s eta 0:00:00
                                              0.0/503.6 kB ? eta -:--:--
     --------------------------------       430.1/503.6 kB 8.9 MB/s eta 0:00:01
     ------------------------------------- 503.6/503.6 kB 10.5 MB/s eta 0:00:00
                                              0.0/119.7 kB ? eta -:--:--
     -------------------------------------- 119.7/119.7 kB 6.8 MB/s eta 0:00:00
                                              0.0/11.3 MB ? eta -:--:--
     -                                        0.5/11.3 MB 11.3 MB/s eta 0:00:01
     ---                                      0.9/11.3 MB 11.0 MB/s eta 0:00:01
     ----                                     1.4/11.3 MB 12.8 MB/s eta 0:00:01
    


[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import evaluate

# Load the BLEU metric
bleu = evaluate.load("sacrebleu")

# Suppose you have:
predictions = ["Xin chào, bạn khỏe không?"]
references = [["Chào bạn, bạn có khỏe không?"]]  # List of lists

# Compute BLEU score
results = bleu.compute(predictions=predictions, references=references)
print(results)


{'score': 26.647313141084275, 'counts': [5, 3, 1, 0], 'totals': [7, 6, 5, 4], 'precisions': [71.42857142857143, 50.0, 20.0, 12.5], 'bp': 0.8668778997501817, 'sys_len': 7, 'ref_len': 8}


In [2]:
from datasets import load_dataset
dataset = load_dataset("nhuvo/MedEV")

README.md: 0.00B [00:00, ?B/s]

d:\STUDY\JAIST\Minor_Project\proj1\MedEV_Evaluate\scripts\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\phamn\.cache\huggingface\hub\datasets--nhuvo--MedEV. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling ba

train.en.txt:   0%|          | 0.00/48.6M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


train.vi.txt:   0%|          | 0.00/61.9M [00:00<?, ?B/s]

...

test.en.new.txt: 0.00B [00:00, ?B/s]

test.vi.new.txt: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/681794 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/17878 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/17920 [00:00<?, ? examples/s]

In [3]:
dataset["test"].to_csv("medEV_test.csv",index=False)

Creating CSV from Arrow format:   0%|          | 0/18 [00:00<?, ?ba/s]

2959896

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
from sacrebleu import corpus_bleu
import torch

# Load the model and tokenizer
model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Load MedEV dataset
dataset = load_dataset("nhuvo/MedEV", split="test")

# Evaluate on a small subset first (e.g. 100 sentences)
subset = dataset.select(range(100))
preds, refs = [], []

for sample in subset:
    input_text = sample["en"]  # English → Vietnamese
    reference = sample["vi"]

    # Tokenize and translate
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model.generate(**inputs, max_length=256, num_beams=5)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

    preds.append(prediction)
    refs.append(reference)

# Compute BLEU
bleu = corpus_bleu(preds, [refs])
print(f"BLEU Score on 100 samples: {bleu.score:.2f}")


KeyError: 'en'

In [4]:
import pandas as pd

# Load CSV
df = pd.read_csv("medEV_test.csv")   # your CSV file
texts = df["text"].tolist()

# Split into English and Vietnamese halves
midpoint = len(texts) // 2
english = texts[:midpoint]
vietnamese = texts[midpoint:]

# Pair them up
paired_data = list(zip(english, vietnamese))

# Convert to DataFrame
aligned_df = pd.DataFrame(paired_data, columns=["en", "vi"])
print(aligned_df.head())

# Save aligned version for later use
aligned_df.to_csv("medev_test_aligned.csv", index=False)


                                                  en  \
0  Knowledge, practices in public health service ...   
1  Describe knowledge, practices in public health...   
2  Methodology: A cross sectional study was used ...   
3  Results: Percentage of card's holders who knew...   
4  Percentage of card's holders who went to the f...   

                                                  vi  
0  Thực trạng kiến thức và thực hành của người có...  
1  Mô tả thực trạng kiến thức, thực hành của ngườ...  
2  Phương pháp: Thiết kế nghiên mô tả cắt ngang đ...  
3  Kết quả: Tỷ lệ người biết được khám chữa bệnh ...  
4  Tỷ lệ người có thẻ BHYT thực hành khám chữa bệ...  


In [14]:
from sacrebleu import corpus_bleu
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Load aligned dataset
aligned_df = pd.read_csv("medev_test_aligned.csv")

model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

preds, refs = [], []

for i in range(50):  # test on first 50 pairs
    src = aligned_df["en"][i]
    tgt = aligned_df["vi"][i]
    
    inputs = tokenizer(src, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model.generate(**inputs, max_length=256, num_beams=5)
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
    #print(pred)
    #print(tgt)

    preds.append(pred)
    refs.append(tgt)

bleu = corpus_bleu(preds, [refs])
print(f"BLEU Score: {bleu.score:.2f}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


BLEU Score: 42.56


In [15]:
from datasets import load_dataset
dataset = load_dataset("nhuvo/MedEV", split="test")
dataset

Dataset({
    features: ['text'],
    num_rows: 17920
})

In [19]:
from sacrebleu import corpus_bleu
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Load aligned dataset
aligned_df = pd.read_csv("medev_test_aligned.csv")

model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

preds, refs = [], []

for i in range(50):  # test on first 50 pairs
    src = aligned_df["vi"][i+50]
    tgt = aligned_df["en"][i+50]
    
    inputs = tokenizer(src, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model.generate(**inputs, max_length=256, num_beams=5)
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
    #print(pred)
    #print(tgt)

    preds.append(pred)
    refs.append(tgt)

bleu = corpus_bleu(preds, [refs])
print(f"BLEU Score: {bleu.score:.2f}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


BLEU Score: 21.38


In [1]:
import transformers
import pandas as pd
import random
from sacrebleu import corpus_bleu
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# --- Load dataset ---
aligned_df = pd.read_csv(r"D:\STUDY\JAIST\Minor_Project\proj1\MedEV_Evaluate\scripts\medev_test_aligned.csv")
print(f"Loaded {len(aligned_df)} sentence pairs.")

# --- Randomly select 50 unique indices ---
random.seed(42)  # for reproducibility (you can remove this line to get different samples each time)
sample_indices = random.sample(range(len(aligned_df)), 50)

# --- Load model ---
model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

preds, refs = [], []

# --- Translation loop ---
for idx in sample_indices:
    src = aligned_df["vi"][idx]   # Vietnamese → English (you can flip if needed)
    tgt = aligned_df["en"][idx]

    inputs = tokenizer(src, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model.generate(**inputs, max_length=256, num_beams=5)
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)

    preds.append(pred)
    refs.append(tgt)

# --- Compute BLEU ---
bleu = corpus_bleu(preds, [refs])
print(f"\nRandomly sampled 50 sentences")
print(f"BLEU Score: {bleu.score:.2f}")


Loaded 8960 sentence pairs.


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Randomly sampled 50 sentences
BLEU Score: 25.38
